# Discord Remote

A remote desktop remote control for Discord applications.



## Server Setup

Creates an express server with some route handling for clicking and typing.


### Remote Activity

The initial command to start the activity within the Discord application.



#### the code

discord remote control?

export TOKENPATH=/Users/briancullinan/.credentials/discord-bot6.txt && export DEFAULT_APPLICATION=1335491680630997034 && npm run import -- "discord remote control"




In [ ]:

const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {interactionsCommands} = importer.import('discord gateway')
const { startServer, closeExpress } = importer.import('express service')
const {doMention, doPrivate} = importer.import('discord llm interactions')

const ALL_COMMANDS = [
  'remote',
]

// bot commands using new API, same names as above but lower-case
async function activityCommands(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)

  interactionsCommands['startActivity'] = startServer.bind(null, ['discord remote.ipynb'])
  interactionsCommands['endActivity'] = closeExpress

  interactionsCommands['promptMention'] = doMention
  interactionsCommands['promptPrivate'] = doPrivate

  cmdDef = {
    'name': 'remote',
    'description': 'Launch the remote',
    'type': 4,
    'handler': 2,
  }
  if(!commands.includes('remote')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'remote')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }

  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}

module.exports = activityCommands





### Express Server

Start the express server that runs the discord activity through the Discord proxy service.



#### the code

discord remote proxy server?

https://1335491680630997034.discordsays.com/.proxy/

ROUTE = /remote

ROOT = true

DEFAULT = true



In [ ]:
const {DEFAULT_APPLICATION} = importer.import('discord configuration')

const BASE_URI = `https://${DEFAULT_APPLICATION}.discordsays.com/.proxy/`

function serveHomepage(req, res, next) {
  if(req.path.length <= 1) {
    let htmlCode = importer.interpret('remote desktop client').code
    htmlCode = htmlCode.replaceAll('{BASE_URI}', BASE_URI)
    htmlCode = htmlCode.replace('<head>', `<head>
      <script type="text/javascript">
      ${importer.interpret('discord client auth code').code
        .replaceAll('{CLIENT_ID}', DEFAULT_APPLICATION)
        .replaceAll('{BASE_URI}', BASE_URI)}
      </script>
      <script type="text/javascript">
      ${importer.interpret('client input remote code').code
        .replaceAll('{CLIENT_ID}', DEFAULT_APPLICATION)
        .replaceAll('{BASE_URI}', BASE_URI)}
      </script>`)
    return res.send(htmlCode)
  }
  return next()
}

module.exports = serveHomepage




### Automation Connector

Connect click and typing callbacks from the client to the remote.ipynb automation jupyter notebook.




#### the code

express automation routes?

reusable parts

ROUTE = /monitors/*


In [ ]:
const {DEFAULT_APPLICATION} = importer.import('discord configuration')
const screenshotMonitor = importer.import('node screenshots')
const mouseMove = importer.import('mouse move')
const mouseClick = importer.import('mouse click')
const sendKeys = importer.import('send keys')

async function getMonitor(req, res, next) {
  let index = parseInt(req.path.replace(/^\/|\/$/gi, '').split('/')[1])
  if(Number.isNaN(index)) {
    index = 0
  }
  let screenshot = await screenshotMonitor(index)
  if(!screenshot) {
    return next()
  }
  res.setHeader('Content-Type', 'image/png')
  res.setHeader('Cache-Control', 'public, max-age=3');
  return res.send(screenshot)
}


let lastClick = 0
let lastX
let lastY
async function doClick(req, res, next) {
  if(!req.authenticated) {
    return res.status(403).send('Forbidden')
  }
  if(Date.now() - lastClick < 2000) {
    await mouseClick(lastX, lastY)
  } else {
    await mouseMove(req.body.x, req.body.y)
  }
  lastClick = Date.now()
  lastX = req.body.x
  lastY = req.body.y
  return res.send('')
}

async function doKeys(req, res, next) {
  if(!req.authenticated) {
    return res.status(403).send('Forbidden')
  }
  sendKeys(req.body.text, req.body.special)
  return res.send('')
}

module.exports = {
  doKeys,
  doClick,
  getMonitor
}



## Client Pages



### Client Homepage

Serve this page to the client to start the whole remote program.



#### the code


remote desktop client?

replace {BASE_URI} with server accessible address


In [ ]:
<html>
<head>
  <meta http-equiv="Content-Security-Policy" content="default-src *;
   img-src * 'self' data: https:; script-src 'self' 'unsafe-inline' 'unsafe-eval' *;
   style-src  'self' 'unsafe-inline' *" />
  <style type="text/css">
    html, body {
      padding: 0;
      margin: 0;
      background: rgb(51, 51, 68);
      color: white;
      font-family: Verdana, Geneva, Tahoma, sans-serif;
    }
    
    h1,
    h2,
    h3 {
      color: white;
      padding: 10px;
      margin: 0;
    }
    
    div.container {
      /*min-height: 1080px;*/
      min-width: 3840px;
      width: auto;
      padding: 20mm 16mm;
      background: #222;
      box-shadow: rgb(0 0 0 / 50%) 2px 3px 10px;
      margin: 20px;
      border-radius: 2px;
      border: 1px solid #666;
      position: relative;
      z-index: 1;
    }
    
    div.livedev {
      position: relative;
      margin: 0 auto;
      max-width: 100%;
      max-height: 460px;
      display:flex;
      padding-bottom: 18.75%;
      position: relative;
    }
    
    div.livedev img {
      width: 33.3%;
      min-width: 33.3%;
      height: 100%;
      position: absolute;
      background-size: 100% contain;
      border: none;
      border-style: none;
      background-repeat: no-repeat;
      left: 0;
    }

    div.livedev img:nth-of-type(2) {
      left: 33.3%;
    }

    div.livedev img:nth-of-type(3) {
      left: 66.6%;
    }
    
    div.livedev p {
      text-align: center;
      padding-bottom: 60%;
    }
    
    div.livedev iframe {
      height: 100%;
      width: 100%;
      position: absolute;
      top: 0;
      left: 0;
      bottom: 0;
      right: 0;
    }

    div.container p {
      display: flex;
      flex-wrap: wrap;
    }

    div.container p input {
      opacity: 0;
      height: 1px;
      width: 1px;
      position: absolute;
      right: 1px;
      bottom: 1px;
    }

    div.container p label,
    div.container p button {
      border: 5px solid transparent;
      border-radius: 10px;
      background: #226;
      position: relative;
      display: inline-block;
      background-clip: padding-box;
      z-index: unset;
      list-style: none;
      text-align: left;
      text-decoration: none;
      color: white;
      min-width: 200px;
      text-align: center;
      cursor: pointer;
      padding: 10px;
      font-size: unset;
    }

    input[id="special-shift"]:checked ~ label[for="special-shift"]
    input[id="special-control"]:checked ~ label[for="special-control"]
    input[id="special-command"]:checked ~ label[for="special-command"]
    input[id="special-alt"]:checked ~ label[for="special-alt"] {
      background-color: white;
      color: #226;
    }

    #sendkeys {
      width: 100%;
    }
    
    
  </style>
</head>
<body>

<div class="container">
  <h1>Live Development</h2>
  <ol class="windows"></ol>
  <div class="livedev">
    <img src="{BASE_URI}monitors/0" />
    <img src="{BASE_URI}monitors/1" />
    <img src="{BASE_URI}monitors/2" />
    <!--<p>
      <script>
        document.write(`<iframe width="560" height="315" allow="autoplay" modestbranding="true" src="https://www.youtube.com/embed/live_stream?channel=UCPaZDuwY1sJOb5l-QHm9mDw&origin=${window.location.origin}&controls=0&rel=0&autoplay=1" frameborder="0" allowfullscreen></iframe>`)
      </script>
    </p>-->
  </div>

  <p>
    <textarea id="sendkeys"></textarea>
    <span>
      <input type="checkbox" id="special-alt" value="alt" />
      <input type="checkbox" id="special-command" value="command" />
      <input type="checkbox" id="special-control" value="control" />
      <input type="checkbox" id="special-shift" value="shift" />
      <button id="special-esc" value="">send</button>
      <button id="special-esc" value="enter">enter</button>
      <button id="special-esc" value="escape">escape</button>
      <button id="special-esc" value="backspace">backspace</button>
      <button id="special-esc" value="delete">delete</button>
      <label for="special-alt">alt</label>
      <label for="special-command">command</label>
      <label for="special-control">control</label>
      <label for="special-shift">shift</label>
    </span>
    <a class="nowButton" target="_blank" href="https://www.youtube.com/channel/UCPaZDuwY1sJOb5l-QHm9mDw">
      My YouTube Channel</a>
  </p>
</div>
</body>
</html>




### Remote Control

Response to page events and send them over Javascript fetch() to the express server.



#### the code

client input remote code?


In [ ]:

document.addEventListener('DOMContentLoaded', (evt) => {

  let queryParams = {}
  let params = (window.location.search.split('?')[1] || '').split('&')
  for(var param in params) {
    if (params.hasOwnProperty(param)) {
      let paramParts = params[param].split('=')
      queryParams[paramParts[0]] = decodeURIComponent(paramParts[1] || "")
    }
  }

  if(queryParams.session) {
    window.session_id = queryParams.session
  }
  
  document.querySelector('.livedev').addEventListener('click', (evt) => {
    debugger
    let localX = evt.clientX - evt.currentTarget.offsetLeft
    let localY = evt.clientY - evt.currentTarget.offsetTop
    fetch('{BASE_URI}click?session=' + window.session_id, {
      method: 'POST',
      headers: {
        "Content-Type": "application/json",
      },
      body: JSON.stringify({
        x: localX / evt.currentTarget.clientWidth,
        y: localY / evt.currentTarget.clientHeight
      })
    })
  })


  function getSpecialKeys() {
    let modifiers = document.querySelectorAll('.container input:checked')
    let result = []
    for(let i = 0; i < modifiers.length; i++) {
      result.push(modifiers[i].value)
    }
    return result
  }


  let currentTimer
  document.querySelector('#sendkeys').addEventListener('keypress', (evt) => {
    if(currentTimer) {
      return
    }
    currentTimer = setTimeout(((target) => {
      fetch('{BASE_URI}keys?session=' + window.session_id, {
        method: 'POST',
        headers: {
          "Content-Type": "application/json",
        },
        body: JSON.stringify({
          text: target.value,
          special: getSpecialKeys()
        })
      })
      target.value = ''
      currentTimer = void 0
    }).bind(null, evt.currentTarget), 1000)
    evt.preventDefault()
  })


  let buttons = document.querySelectorAll('.container button')
  for(let i = 0; i < buttons.length; i++) {
    buttons[i].addEventListener('click', (evt) => {
      let target = document.querySelector('#sendkeys')
      let keys = getSpecialKeys()
      if(evt.currentTarget.value)
        keys.push(evt.currentTarget.value)
      fetch('{BASE_URI}keys?session=' + window.session_id, {
        method: 'POST',
        headers: {
          "Content-Type": "application/json",
        },
        body: JSON.stringify({
          text: target.value,
          special: keys
        })
      })
      target.value = ''
      evt.preventDefault()
    })
  }



  setInterval(() => {
    // TODO: refresh images
    let images = document.querySelectorAll('.livedev > img')
    for(let i = 0; i < images.length; i++) {
      if(images[i].src && !images[i].getAttribute('original-src')) {
        images[i].setAttribute('original-src', images[i].src)
      }
      let originalImage = images[i].getAttribute('original-src')
      let newImage = originalImage.replace(/\?.*/, '') + '?t=' + Date.now()
      // set background to previous image
      if(images[i].src) {
        images[i].style.backgroundImage = "url('" + encodeURI(images[i].src) + "')"
      }
      setTimeout(() => {
        images[i].src = newImage
      }, 100)
    }
  }, 5000)
})

